### 1. Preprocess and split all books into segments

In [7]:
import json
import sys
from pathlib import Path

sys.path.append(str(Path.cwd().parent))
from book_segmenting import TextSegmenter
from book_preprocessing import TxtBookPreprocessor
from utils import SEGMENT_DIR, BOOK_DIR

SEGMENT_CHARS_MIN = 150
SEGMENT_CHARS_MAX = 500
segment_dir = SEGMENT_DIR / "batch2"

preprocessor = TxtBookPreprocessor()
segmenter = TextSegmenter(chunk_size=(SEGMENT_CHARS_MIN, SEGMENT_CHARS_MAX))

for f in segment_dir.glob("*.json"):
    f.unlink()
for book in BOOK_DIR.glob("*.txt"):
    with open(book, "r", encoding="utf-8-sig") as f:
        book_content = f.read()
    book_slug = book.stem
    chunks = segmenter.segment_text(preprocessor.clean_text(book_content))
    with open(segment_dir / f"{book_slug}.json", "w", encoding="utf-8") as f:
        json.dump(chunks, f, indent=2)

### 2. Sample N segments from all unused yet

In [8]:
import sys
from pathlib import Path

sys.path.append(str(Path.cwd().parent))
from utils import SEGMENT_DIR, TO_ANNOTATE_DIR, BOOK_META_DIR
from dataset_small.segment_sampling import SegmentSampler

N = 10000
BATCH_NAME = "batch_10k"
MAX_PER_BOOK = 3
SEED = 42
segment_dir = SEGMENT_DIR / "batch2"

sampler = SegmentSampler(segment_dir, BOOK_META_DIR, TO_ANNOTATE_DIR, SEED)
sample = sampler.sample_balanced(N, BATCH_NAME, max_per_book=MAX_PER_BOOK)
sampler.print_status()

Available: 610668 segments
  Science Fiction: only 90 available (wanted 1428)
  Fiction: only 603 available (wanted 1428)
  Western: only 66 available (wanted 1428)
  Fantasy: only 147 available (wanted 1428)
  Mystery: only 222 available (wanted 1428)
  History: only 57 available (wanted 1428)
  Travel: only 15 available (wanted 1428)

Saved 10000 segments to /home/terra/Projects/vis-desc/modules/lab/data/to-annotate/batch_10k.json
=== CORPUS STATUS ===
Books: 400
Total segments: 611140
Used segments: 500
Available: 610640

By genre:
  Fiction: 315528 (201 books)
  Mystery: 106695 (74 books)
  Fantasy: 60682 (49 books)
  Western: 45875 (22 books)
  Science Fiction: 43143 (30 books)
  History: 32717 (19 books)
  Travel: 6500 (5 books)


In [ ]:
import json
import collections
from pathlib import Path
import matplotlib.pyplot as plt
from utils import BOOK_META_DIR, DATA_DIR

genre_counts = collections.Counter()
for meta_file in BOOK_META_DIR.glob("*.json"):
    try:
        with open(meta_file, "r", encoding="utf-8") as f:
            data = json.load(f)
        g = data.get("genre") or "Unknown"
        if isinstance(g, str):
            g = g.strip() or "Unknown"
        else:
            g = "Unknown"
        if g.lower() == "science fiction":
            g = "Sci-Fi"
        genre_counts[g] += 1
    except Exception:
        pass

if not genre_counts:
    print("No genre data found for Matplotlib chart.")
else:
    genres, counts = zip(*genre_counts.most_common())
    total = sum(counts)
    plt.rcParams.update({"font.size": 42, "axes.titlesize": 48, "legend.fontsize": 32})

    fig, ax = plt.subplots(figsize=(20, 14), dpi=120)

    def autopct_fmt(pct):
        return ("%0.1f%%" % pct) if pct >= 1 else ""

    wedges, texts, autotexts = ax.pie(
        counts,
        labels=None,  # legend instead of slice text
        autopct=autopct_fmt,
        startangle=90,
        counterclock=False,
        pctdistance=0.70,
        textprops={"color": "#111", "fontsize": 44},
        wedgeprops={"linewidth": 2.5, "edgecolor": "white"},
    )

    for t in autotexts:
        t.set_fontsize(44)
        t.set_fontweight("bold")

    from matplotlib.patches import Patch

    legend_patches = [
        Patch(facecolor=w.get_facecolor(), edgecolor="white", label=f"{g} ({c})")
        for w, g, c in zip(wedges, genres, counts)
    ]

    fig.subplots_adjust(left=0, right=0.80, top=1, bottom=0)

    leg = fig.legend(
        handles=legend_patches,
        loc="center left",
        bbox_to_anchor=(0.7, 0.5),
        frameon=True,
        title_fontsize=36,
        borderpad=0.4,
        labelspacing=0.9,
        handlelength=1.6,
        handleheight=1.1,
    )
    frame = leg.get_frame()
    frame.set_alpha(0.18)
    frame.set_edgecolor("#bcbcbc")
    frame.set_linewidth(1.2)
    frame.set_facecolor("#ffffff")

    ax.set_aspect("equal")

    pie_path = DATA_DIR / "genre_distribution.png"
    fig.savefig(pie_path, bbox_inches="tight", pad_inches=0, transparent=True)
    print("Saved pie to", pie_path)
    plt.show()
    print("Total books counted:", total)

print("Done.")